# Bayesian MBG Predictions (batched)

In [68]:
import functions

import pymc as pm

import os
import pickle
import numpy as np
import arviz as az
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler
import pytensor.tensor as at

import uuid
import pytensor.tensor as at
import scipy.stats as stats


## Geostatistical Modeling

### Parameters

In [41]:
#Load report
with open('temp_files/report/report.pkl', 'rb') as pickle_file:
    report = pickle.load(pickle_file)

In [42]:
#Define the directory where the pickle files are stored
pickle_dir = 'temp_files'

target_indicator = report['Target Indicator']

# Get the number of CPU cores to max out the machine in the traning stage
num_cores = os.cpu_count()

print(f"Number of CPU cores: {num_cores}")

Number of CPU cores: 8


### Load target and covariates

In [46]:
df = pd.read_pickle('temp_files/selected_features.pkl')

In [47]:
selected_features = df.columns.to_list()

# Remove target_values and others
remove_list = [target_indicator, 'geometry', 'grid_id']

# Remove elements in remove_list from main_list
selected_features = [item for item in selected_features if item not in remove_list]

In [48]:
# Load coordinates from the pickle file
with open(os.path.join(pickle_dir, 'coordinates.pkl'), 'rb') as f:
    coordinates = pickle.load(f)

# Load coordinates for observed rows from the pickle file
with open(os.path.join(pickle_dir, 'coordinates_observed.pkl'), 'rb') as f:
    coordinates_observed = pickle.load(f)

In [49]:
#Only rows with observed target indicator
df1 = df[~df[target_indicator].isnull()]

In [50]:
# Covariate matrix
X = df1[selected_features].values

# Series with the target variable observed
response = df1[target_indicator].values

In [51]:
# Standardize features and response
X = (X - X.mean(axis=0)) / X.std(axis=0)
response = (response - response.mean()) / response.std()

### Recreate the saved model

In [52]:
# After fitting the model
trace_filename = 'model_trace.nc'

In [53]:
#Load the model
idata = az.from_netcdf(trace_filename)

# Re-create the model
with pm.Model() as model:
    # Re-create the model's priors and components
    beta = pm.Normal('beta', mu=0, sigma=1, shape=len(selected_features))
    sigma = pm.HalfNormal('sigma', sigma=1)
    ls = pm.HalfCauchy('ls', beta=1)

    D = np.sqrt(((coordinates_observed[:, None, :] - coordinates_observed[None, :, :])**2).sum(axis=-1))

    K = pm.gp.cov.Matern52(2, ls=ls)
    gp = pm.gp.Latent(cov_func=K)
    f = gp.prior('f', X=coordinates_observed)

    mu = pm.math.dot(X, beta) + f
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=response)

## Generating predictions for all grids

In [63]:
df2 = df 

df2 = df2.sample(400) #temporary to make some tests

df2.reset_index(drop=True, inplace=True)

# Covariate matrix
X_new = df2[selected_features].values

# Standardize the new data using the same scaler fitted on the observed data
X_new = (X_new - X_new.mean(axis=0)) / X_new.std(axis=0)

# Load coordinates for the new locations
with open(os.path.join(pickle_dir, 'coordinates_unobserved.pkl'), 'rb') as f:
    coordinates_new = pickle.load(f)

# Ensure the scaler is fitted on the same data
#scaler = StandardScaler().fit(df1[selected_features].values)
#X_new = scaler.transform(df2[selected_features].values)

### Check if the covariance matrix is PSD
- PSD: Positive Semi-Definitive

In [64]:
#This function is key to diagnose what is going on inside the Gaussian process

def diagnose_covariance_matrix(cov, jitter=1e-6):
    """
    Diagnose potential issues with a covariance matrix and suggest possible remedies.

    Parameters:
    ----------
    cov : np.ndarray or pytensor.tensor
        The covariance matrix to diagnose.
    jitter : float, optional
        The amount of jitter to add to the diagonal of the covariance matrix for stabilization.

    Returns:
    -------
    None
    """

    # Convert pytensor tensor to numpy array for diagnosis if necessary
    if isinstance(cov, at.TensorVariable):
        # Use pm.draw to evaluate the tensor as a NumPy array
        cov = pm.draw(cov)

    # Check for symmetry
    if not np.allclose(cov, cov.T):
        print("Warning: Covariance matrix is not symmetric.")
    else:
        print("Covariance matrix is symmetric.")

    # Check for positive semi-definiteness using eigenvalues
    eigvals = np.linalg.eigvalsh(cov)
    if np.all(eigvals >= 0):
        print("Covariance matrix is positive semi-definite (PSD).")
    elif np.all(eigvals > 0):
        print("Covariance matrix is positive definite (PD).")
    else:
        print("Covariance matrix is not positive semi-definite (non-PSD).")
        print("Eigenvalues:")
        print(eigvals)

    # Check for small or negative eigenvalues
    if np.any(eigvals < 0):
        print("There are negative eigenvalues, indicating non-PSD matrix.")
    elif np.any(eigvals == 0):
        print("There are zero eigenvalues, indicating the matrix is singular or nearly singular.")
    if np.any(eigvals < jitter):
        print("Some eigenvalues are smaller than the jitter value. Consider increasing jitter.")

    # Check the condition number (ratio of max to min eigenvalue)
    cond_number = np.linalg.cond(cov)
    print(f"Condition number of the matrix: {cond_number:.2e}")
    if cond_number > 1e10:
        print("Warning: Covariance matrix is ill-conditioned (large condition number).")
        print("Consider regularization or using a different covariance function.")

    # Suggest adding jitter and re-check PSD
    cov_with_jitter = cov + jitter * np.eye(cov.shape[0])
    eigvals_with_jitter = np.linalg.eigvalsh(cov_with_jitter)
    if np.all(eigvals_with_jitter >= 0):
        print("Adding jitter made the covariance matrix positive semi-definite.")
    else:
        print("Even after adding jitter, the matrix is still not positive semi-definite.")

    # Check for numerical issues using Cholesky decomposition
    try:
        np.linalg.cholesky(cov)
        print("Cholesky decomposition succeeded: Covariance matrix is positive definite.")
    except np.linalg.LinAlgError:
        print("Cholesky decomposition failed: Covariance matrix is not positive definite.")

    print("\nDiagnosis Complete.")

def generate_predictions(model, coordinates_new, X_new, idata, initial_jitter=1e-6, max_attempts=5):
    """
    Generate predictions for new data using a Gaussian Process model.

    Parameters:
    ----------
    model : pm.Model
        The PyMC model object that contains the Gaussian Process.
    coordinates_new : np.ndarray
        An array of coordinates for the new data points where predictions are needed.
    X_new : np.ndarray
        The covariate matrix for the new data points.
    idata : az.InferenceData
        The InferenceData object containing posterior samples from the fitted model.
    initial_jitter : float, optional
        The initial jitter value to add to the covariance matrix to ensure positive definiteness.
    max_attempts : int, optional
        Maximum number of attempts to find a stable jitter value.

    Returns:
    -------
    np.ndarray
        An array of mean predictions for the new data points.
    """

    with model:
        for attempt in range(max_attempts):
            try:
                jitter = initial_jitter * (10 ** attempt)
                unique_name = "f_pred_" + str(uuid.uuid4())

                # Generate the conditional GP with added jitter to the covariance matrix
                f_pred = gp.conditional(unique_name, coordinates_new, jitter=jitter)

                # Compute the mean of the beta samples
                beta_mean = idata.posterior['beta'].mean(dim=("chain", "draw")).values

                # Predictive mean
                mu_pred = pm.math.dot(X_new, beta_mean) + f_pred

                # Create the covariance matrix using PyMC's Matern32
                cov = pm.gp.cov.Matern32(coordinates_new.shape[1], ls=1.0)(coordinates_new)

                # Add jitter using Pytensor's identity matrix
                cov += jitter * at.eye(cov.shape[0])

                # Symmetrize the covariance matrix to ensure symmetry
                cov = (cov + cov.T) / 2

                # Check cov_matrix before predictions
                diagnose_covariance_matrix(cov)
                
                # Check for positive definiteness using Cholesky decomposition
                _ = at.slinalg.cholesky(cov)

                # If successful, proceed with prediction
                pred_samples = pm.sample_posterior_predictive(idata, var_names=[unique_name], return_inferencedata=True)
                return pred_samples.posterior_predictive[unique_name].mean(axis=0)

            except Exception as e:
                if attempt == max_attempts - 1:
                    raise ValueError(f"The covariance matrix is not positive semi-definite even after {max_attempts} attempts with increasing jitter.") from e


In [66]:
#Dataframe to store predictions and other values for uncertainty calculations
#df3 = pd.DataFrame(df2[['grid_id', 'mpi']].head(0))

In [59]:
#Pick-up productions were we left of. 
with open('temp_files/predictions.pkl', 'rb') as pickle_file:
    df3 = pickle.load(pickle_file)

In [69]:
start_position = df3.index.max() + 1

In [67]:
# Assuming 95% confidence level, change alpha for different confidence levels
alpha = 0.05
z_score = stats.norm.ppf(1 - alpha/2)

In [70]:
step = 50

In [ ]:
for i in range(start_position, len(df2), step):

    r = range(i, i+step)

    print(r)
    
    X_new_i = X_new[r]
    
    coordinates_new_i = coordinates_new[r]
    
    predictions_i = generate_predictions(model, coordinates_new_i, X_new_i, idata)
    
    # Extracting relevant data
    dfi = df2.loc[r][['grid_id', target_indicator]]
    dfpi = pd.DataFrame(predictions_i)
    
    # Calculating mean, standard deviation, and other statistics
 
    dfi[target_indicator] = dfpi.mean().values  # Mean prediction for each observation
    dfi['std'] = dfpi.std().values              # Standard deviation for each observation
    dfi['max'] = dfpi.max().values              # Max prediction for each observation
    dfi['min'] = dfpi.min().values              # Min prediction for each observation
     
    # Calculating the margin of error for the confidence interval
    dfi['standard_error'] = dfi['std'] / (len(dfpi)**0.5)
    dfi['margin_of_error'] = z_score * dfi['standard_error']
    
    # Calculating the confidence interval
    dfi['ci_lower'] = dfi[target_indicator] - dfi['margin_of_error']
    dfi['ci_upper'] = dfi[target_indicator] + dfi['margin_of_error']
    
    df3 = pd.concat([df3, dfi])

    #Save predictions so far
    df3.to_pickle('temp_files/predictions.pkl')

range(0, 50)
Covariance matrix is symmetric.
Covariance matrix is positive semi-definite (PSD).
Condition number of the matrix: 1.22e+06
Adding jitter made the covariance matrix positive semi-definite.
Cholesky decomposition succeeded: Covariance matrix is positive definite.

Diagnosis Complete.


Sampling: [f_pred_06bbf3f9-2fd3-4d37-b3b2-134f6f638d49]


Output()

Covariance matrix is symmetric.
Covariance matrix is positive semi-definite (PSD).
Condition number of the matrix: 9.46e+05
Adding jitter made the covariance matrix positive semi-definite.
Cholesky decomposition succeeded: Covariance matrix is positive definite.

Diagnosis Complete.


Sampling: [f_pred_f0d6651d-21da-45c2-91e7-011e91ae7bfc]


...

Sampling: [f_pred_9b16cadc-a1fc-46d9-be50-80ba57c8c949]


Output()

Covariance matrix is symmetric.
Covariance matrix is positive semi-definite (PSD).
Condition number of the matrix: 3.74e+04
Adding jitter made the covariance matrix positive semi-definite.
Cholesky decomposition succeeded: Covariance matrix is positive definite.

Diagnosis Complete.


Sampling: [f_pred_3c75efc0-f8c6-464b-9c78-240971c7662c]


Output()

### Revert normalization and transform

In [22]:
df3[target_indicator+'_untransformed'] = df3[target_indicator] #Security Copy

#Revert standardization
df3[target_indicator+'_denormalized'] = functions.revert_standardization(df3[target_indicator+'_untransformed'], 
                                                                         report['Target mean'], 
                                                                         report['Target std'])

#Revert transformation
df3[target_indicator] = functions.revert_transformation(df3[target_indicator+'_denormalized'], 
                                report['Transformation Applied'],
                                report['Target mean'], 
                                report['Target std'], 
                               report['Lambda']).round(4)

### Uncertainty Quantification

#### Credible Intervals

Evaluate the width of the credible intervals for predictions. Narrower intervals indicate higher precision, but they should still encompass the true values.

Width of the Confidence Interval (ci_width):

    This represents the range between the upper and lower bounds of the confidence interval.

Absolute Confidence Interval (absolute_confidence_interval):

    This is half the width of the confidence interval. It's the margin of error, representing how far the estimate could be from the true population parameter, in absolute terms.

In [23]:
# Adding a column to represent the width of the confidence interval
df3['ci_width'] = df3['ci_upper'] - df3['ci_lower']

# Adding a column to represent the absolute confidence interval (absolute margin of error)
df3['absolute_confidence_interval'] = df3['ci_width'] / 2 #Absolute margin of error

In [24]:
df3['standardized_uncertainty'] = (df3['absolute_confidence_interval'] - df3['absolute_confidence_interval'].mean())  / df3['absolute_confidence_interval'].std()

In [33]:
df3.to_pickle('temp_files/predictions_final.pkl')